In [7]:
### This file contains code to reproduce the volatility prediction experiments found in https://arxiv.org/abs/2208.08401 
source("DtACI.R")
source("MVP.R")
source("AgACI.R")
library(rugarch)

In [2]:
### Compute sequence of conformity scores by fitting Garch(garchP,garchQ) model. 
### Returns is a sequence of stock market returns, lookback specifies the dataset \{R_s\}_{t-lookback+1 \leq s \leq t-1}
### used to fit the model and make predictions at time t, startup specifies the first time we make a prediction set,
### badscores = FALSE means we use the conformity scores S_t := |V_t- \hat{\sigma}^2_t|/\hat{\sigma}^2_t else we use
### S_t := |V_t- \hat{\sigma}^2_t|
garchConformalForcastingComputeScores <- function(returns,lookback=1250,garchP=1,garchQ=1,startUp = 100,badScores=FALSE){
  T <- length(returns)
  startUp <- max(startUp,lookback)
  garchSpec <- ugarchspec(mean.model=list(armaOrder = c(0, 0),include.mean=FALSE),variance.model=list(model="sGARCH",garchOrder=c(1,1)),distribution.model="norm")
  scores <- rep(0,T-startUp+1)
  sigmaSeq <- rep(0,T-startUp+1)
  
  for(t in startUp:T){
    garchFit <- ugarchfit(garchSpec, returns[(t-lookback+1):(t-1) ],solver="hybrid")
    sigmaNext <- sigma(ugarchforecast(garchFit,n.ahead=1))
    if(!badScores){
      scores[t-startUp + 1] <- abs(returns[t]^2- sigmaNext^2)/sigmaNext^2
    }else{
      scores[t-startUp + 1] <- abs(returns[t]^2- sigmaNext^2)
    }
    sigmaSeq[t-startUp + 1] <- sigmaNext^2
    
  }
  return(list(scores,sigmaSeq))
}

### Use a binary search to find the lowest quantile of recentScores that is above curScore
### Epsilon gives numerical error tolerance in the binary search
findBeta <- function(recentScores,curScore,epsilon=0.001){
  top <- 1
  bot <- 0
  mid <- (top+bot)/2
  while(top-bot > epsilon){
    if(quantile(recentScores,1-mid)>curScore){
      bot <- mid
      mid <- (top+bot)/2
    }else{
      top <- mid
      mid <- (top+bot)/2
    }
  }
  return(mid)
}

### Given a sequence of conformity scores compute the corresponding values for beta_t
### Epsilon gives numerical error tolerance in a binary search
garchConformalForcastingComputeBetas <- function(scores,lookback=Inf,epsilon=0.001){
  T <- length(scores)
  ### Initialize data storage variables
  betaSeq <- rep(0,T-1)
  
  for(t in 2:T){
    recentScores <- scores[max(t- lookback,1):(t-1)]
    betaSeq[t-1] <- findBeta(recentScores,scores[t],epsilon)
  }
  
  return(betaSeq)
}


In [3]:
### Get stock market price data
amd <- read.csv("MajorDailyReturns/AMDDailyPrices.csv",header=TRUE)
nividia <- read.csv("MajorDailyReturns/NividiaDailyPrices.csv",header=TRUE)
blackberry <- read.csv("MajorDailyReturns/BlackberryDailyPrices.csv",header=TRUE)
fannie <- read.csv("MajorDailyReturns/FannieMaeDailyPrices.csv",header=TRUE)
pricesAMD <- rev(amd$Open)
returnsAMD <- sapply(2:length(pricesAMD),function(x){(pricesAMD[x] - pricesAMD[x-1])/pricesAMD[x-1]})
returnsAMD <- returnsAMD[(length(returnsAMD)-7500):length(returnsAMD)]
pricesBlackberry<- rev(blackberry$Open)
returnsBlackberry <- sapply(2:length(pricesBlackberry),function(x){(pricesBlackberry[x] - pricesBlackberry[x-1])/pricesBlackberry[x-1]})
pricesNividia <- rev(nividia$Open)
returnsNividia <- sapply(2:length(pricesNividia),function(x){(pricesNividia[x] - pricesNividia[x-1])/pricesNividia[x-1]})
pricesFannie <- rev(fannie$Open)
returnsFannie <- sapply(2:length(pricesFannie),function(x){(pricesFannie[x] - pricesFannie[x-1])/pricesFannie[x-1]})
returnsFannie <- returnsFannie[(length(returnsFannie)-7500):length(returnsFannie)]

### Compute all conformity scores
lookback <- 1250
amdScores <- garchConformalForcastingComputeScores(returnsAMD,lookback=lookback)
blackberryScores <- garchConformalForcastingComputeScores(returnsBlackberry,lookback=lookback)
nividiaScores <- garchConformalForcastingComputeScores(returnsNividia,lookback=lookback)
fannieScores <- garchConformalForcastingComputeScores(returnsFannie,lookback=lookback)

amdBadScores <- garchConformalForcastingComputeScores(returnsAMD,lookback=lookback,badScores=TRUE)
blackberryBadScores <- garchConformalForcastingComputeScores(returnsBlackberry,lookback=lookback,badScores=TRUE)
nividiaBadScores <- garchConformalForcastingComputeScores(returnsNividia,lookback=lookback,badScores=TRUE)
fannieBadScores <- garchConformalForcastingComputeScores(returnsFannie,lookback=lookback,badScores=TRUE)

### Compute betas
amdBetas <- garchConformalForcastingComputeBetas(amdScores[[1]],lookback=lookback)
blackberryBetas <- garchConformalForcastingComputeBetas(blackberryScores[[1]],lookback=lookback)
nividiaBetas <- garchConformalForcastingComputeBetas(nividiaScores[[1]],lookback=lookback)
fannieBetas <- garchConformalForcastingComputeBetas(fannieScores[[1]],lookback=lookback)

amdBadBetas <- garchConformalForcastingComputeBetas(amdBadScores[[1]],lookback=lookback)
blackberryBadBetas <- garchConformalForcastingComputeBetas(blackberryBadScores[[1]],lookback=lookback)
nividiaBadBetas <- garchConformalForcastingComputeBetas(nividiaBadScores[[1]],lookback=lookback)
fannieBadBetas <- garchConformalForcastingComputeBetas(fannieBadScores[[1]],lookback=lookback)

In [14]:
### Run DtACI
alpha <- 0.1
gammaGrid <- c(0.001,0.002,0.004,0.008,0.0160,0.032,0.064,0.128)
Keps <- 2.12
amdStableGridGammas <- conformalAdaptStable(amdBetas,alpha,gammaGrid,sigma=1/1000,eta=2.72)
blackberryStableGridGammas <- conformalAdaptStable(blackberryBetas,alpha,gammaGrid,sigma=1/1000,eta=2.72)
nividiaStableGridGammas <- conformalAdaptStable(nividiaBetas,alpha,gammaGrid,sigma=1/1000,eta=2.72)
fannieStableGridGammas <- conformalAdaptStable(fannieBetas,alpha,gammaGrid,sigma=1/1000,eta=2.72)

amdStableGridGammasBad <- conformalAdaptStable(amdBadBetas,alpha,gammaGrid,sigma=1/1000,eta=2.72)
blackberryStableGridGammasBad <- conformalAdaptStable(blackberryBadBetas,alpha,gammaGrid,sigma=1/1000,eta=2.72)
nividiaStableGridGammasBad <- conformalAdaptStable(nividiaBadBetas,alpha,gammaGrid,sigma=1/1000,eta=2.72)
fannieStableGridGammasBad <- conformalAdaptStable(fannieBadBetas,alpha,gammaGrid,sigma=1/1000,eta=2.72)

### Run MVP
amdMVP <- MVP(1-amdBetas,alpha,Keps = Keps)
blackberryMVP <- MVP(1-blackberryBetas,alpha,Keps = Keps)
nividiaMVP <- MVP(1-nividiaBetas,alpha,Keps = Keps)
fannieMVP <- MVP(1-fannieBetas,alpha,Keps = Keps)

amdMVPBad <- MVP(1-amdBadBetas,alpha,Keps = Keps)
blackberryMVPBad <- MVP(1-blackberryBadBetas,alpha,Keps = Keps)
nividiaMVPBad <- MVP(1-nividiaBadBetas,alpha,Keps = Keps)
fannieMVPBad <- MVP(1-fannieBadBetas,alpha,Keps = Keps)

### Run AgACI
amdAgACI <- agaci(amdBetas,alpha,gammaGrid)
blackberryAgACI <- agaci(blackberryBetas,alpha,gammaGrid)
nividiaAgACI <- agaci(nividiaBetas,alpha,gammaGrid)
fannieAgACI <- agaci(fannieBetas,alpha,gammaGrid)

amdAgACIBad <- agaci(amdBadBetas,alpha,gammaGrid)
blackberryAgACIBad <- agaci(blackberryBadBetas,alpha,gammaGrid)
nividiaAgACIBad <- agaci(nividiaBadBetas,alpha,gammaGrid)
fannieAgACIBad <- agaci(fannieBadBetas,alpha,gammaGrid)

In [15]:
### Code to make plots

library(tidyr)
library(dplyr)
library(cowplot)

localMean <- function(seq,index,bandwidth,bottom=300){
  bottomIndex <- max(1,index-bottom+1)
  weights <- (bandwidth)^(bottomIndex:index)
  weights <- weights/sum(weights)
  return(sum(weights*seq[bottomIndex:index]))
}

makeCoveragePlot <- function(errMat,dates,alpha=0.1,smoothingWidth=500,cols = c("dtaci","agaci","mvp","split"),
                     xlabel = "Time",ylabel = "Coverage",colors=c(4,2,3,1),legendNames=cols,timeCutoff=0,lineSize=4,lineTyping=FALSE, alphaC = 0.8){
  T <- nrow(errMat)
  if(is.null(dates)){
      dates <- 1:T
  }
  errMatSmooth <- matrix(0,nrow=nrow(errMat),ncol=ncol(errMat))
  if(smoothingWidth>0){
    for(i in 1:length(cols)){
      errMatSmooth[,i] <- 1-sapply(1:T,function(x){localMean(errMat[,i],x,bandwidth=1,bottom=smoothingWidth)})
    }
  }
    
  colnames(errMatSmooth) <- cols
  finalMat <- as.data.frame(errMatSmooth)
  finalMat$Time <- dates
  finalMat <- pivot_longer(finalMat,cols=1:length(cols)) 
  colnames(finalMat) = c("Time","Method","Coverage")
  if(timeCutoff>0){
     finalMat <- finalMat[-(1:timeCutoff),]
  }
  if(lineTyping){
      p <- ggplot(finalMat,aes(x=Time, y=Coverage, group=Method,color=Method,linetype=Method))
  }else{
      p <- ggplot(finalMat,aes(x=Time, y=Coverage, group=Method,color=Method))
  }
  p <- p + geom_line(alpha=alphaC,size=lineSize) 
  p <- p + xlab(xlabel) + ylab(ylabel)  
  p <- p +  theme_bw() + theme(axis.text=element_text(size=40),axis.title=element_text(size=50,face="plain")) 
  p <- p + geom_hline(yintercept=1-alpha, linetype="dashed", color = "black",size = 4) 
  for(i in 1:ncol(errMat)){
     p <- p + geom_hline(yintercept=1-mean(errMat[,i]), linetype="dashed", color = colors[i],size = 4)   
  }
  return(p)
}


In [16]:
library(gridExtra)
library(ggplot2)
### Plot some of the results
allDates <- list(nividia$Date[1:length(nividiaStableGridGammas[[1]])],
                 amd$Date[1:length(amdStableGridGammas[[1]])],
                 blackberry$Date[1:length(blackberryStableGridGammas[[1]])],
                 fannie$Date[1:length(fannieStableGridGammas[[1]])])
for(d in 1:length(allDates)){
  allDates[[d]] <- rev(allDates[[d]])
  allDates[[d]] <- as.Date(allDates[[d]], '%m/%d/%y')
}

errSeqsAMDBad <- cbind(amdStableGridGammasBad[[6]],amdStableGridGammasBad[[3]],amdMVPBad[[2]],amdAgACIBad[[2]])
errSeqsBBBad <- cbind(blackberryStableGridGammasBad[[6]],blackberryStableGridGammasBad[[3]],blackberryMVPBad[[2]],blackberryAgACIBad[[2]])
errSeqsNvidiaBad <- cbind(nividiaStableGridGammasBad[[6]],nividiaStableGridGammasBad[[3]],nividiaMVPBad[[2]],nividiaAgACIBad[[2]])
errSeqsFannieBad <- cbind(fannieStableGridGammasBad[[6]],fannieStableGridGammasBad[[3]],fannieMVPBad[[2]],fannieAgACIBad[[2]])

amdPlotBad <- makeCoveragePlot(errSeqsAMDBad,allDates[[2]],alpha,cols=c("DtACI","Constant Baseline","MVP","AgACI"),colors=c(4,7,3,2),timeCutoff=500)
amdPlotBad <- amdPlotBad + scale_colour_manual(name="Method",values = c("DtACI"=4,"AgACI"=2,"MVP"=3,"Constant Baseline"=7),
                                                             labels=c("DtACI","AgACI","MVP","Constant Baseline"),
                                                             aesthetics = c("color")) + ylim(0.37,1) +
                    theme(axis.text.x = element_blank(),axis.ticks.x = element_blank(),axis.title.x = element_blank(),
                         axis.text.y = element_blank(),axis.ticks.y = element_blank(),axis.title.y = element_blank()) +
                    theme(legend.key.width = unit(1,"cm"))
bbPlotBad <- makeCoveragePlot(errSeqsBBBad,allDates[[3]],alpha,cols=c("DtACI","Constant Baseline","MVP","AgACI"),colors=c(4,7,3,2),timeCutoff=500)
bbPlotBad <- bbPlotBad + scale_colour_manual(name="Method",values = c("DtACI"=4,"AgACI"=2,"MVP"=3,"Constant Baseline"=7),
                                                             labels=c("DtACI","AgACI","MVP","Constant Baseline"),
                                                             aesthetics = c("color"))  + ylim(0.37,1)
nvidiaPlotBad <- makeCoveragePlot(errSeqsNvidiaBad,allDates[[1]],alpha,cols=c("DtACI","Constant Baseline","MVP","AgACI"),colors=c(4,7,3,2),timeCutoff=500)
nvidiaPlotBad <- nvidiaPlotBad + scale_colour_manual(name="Method",values = c("DtACI"=4,"AgACI"=2,"MVP"=3,"Constant Baseline"=7),
                                                             labels=c("DtACI","AgACI","MVP","Constant Baseline"),
                                                             aesthetics = c("color")) + ylim(0.37,1) +
                            theme(axis.text.x = element_blank(),axis.ticks.x = element_blank(),axis.title.x = element_blank())
fanniePlotBad <- makeCoveragePlot(errSeqsFannieBad,allDates[[4]],alpha,cols=c("DtACI","Constant Baseline","MVP","AgACI"),colors=c(4,7,3,2),timeCutoff=500)
fanniePlotBad <- fanniePlotBad + scale_colour_manual(name="Method",values = c("DtACI"=4,"AgACI"=2,"MVP"=3,"Constant Baseline"=7),
                                                             labels=c("DtACI","AgACI","MVP","Constant Baseline"),
                                                             aesthetics = c("color")) + ylim(0.37,1)  + 
                            theme(axis.text.y = element_blank(),axis.ticks.y = element_blank(),axis.title.y = element_blank())

legend <- get_legend(amdPlotBad+theme(legend.position = "top",legend.text=element_text(size=60),legend.title=element_blank(),
                                     legend.key.width = unit(3,"line")))
blankplot <- ggplot()+geom_blank(aes(1,1)) + cowplot::theme_nothing()

nvidiaPlotBad <- nvidiaPlotBad +  annotate('text',x=as.Date("2005-06-01"), y=0.99, label="Nvidia",size=18,fontface="bold") + theme(legend.position = "none")
amdPlotBad <- amdPlotBad  +  annotate('text',x=as.Date("1997-10-20"), y=0.99, label="AMD",size=18,fontface="bold") + theme(legend.position = "none")
bbPlotBad <- bbPlotBad +  annotate('text',x=as.Date("2006-09-01"), y=0.99, label="BlackBerry",size=18,fontface="bold") + theme(legend.position = "none")
fanniePlotBad <- fanniePlotBad +  annotate('text',x=as.Date("2000-02-20"), y=0.99, label="Fannie Mae",size=18,fontface="bold") + theme(legend.position = "none")

options(repr.plot.width=25, repr.plot.height=15)

pdf("Figures/VolatilityCoverageBad.pdf", width=30, height = 20)
grid.arrange(legend,blankplot,nvidiaPlotBad,amdPlotBad,bbPlotBad,fanniePlotBad,ncol=2,widths=c(3.5,3),heights = c(0.4,2,2.5))
dev.off()

Warning message in get_plot_component(plot, "guide-box"):
“Multiple components found; returning the first one. To return all, use `return_all = TRUE`.”


pdf 
  2

In [17]:
errSeqsAMD <- cbind(amdStableGridGammas[[6]],amdStableGridGammas[[3]],amdMVP[[2]],amdAgACI[[2]])
errSeqsBB <- cbind(blackberryStableGridGammas[[6]],blackberryStableGridGammas[[3]],blackberryMVP[[2]],blackberryAgACI[[2]])
errSeqsNvidia <- cbind(nividiaStableGridGammas[[6]],nividiaStableGridGammas[[3]],nividiaMVP[[2]],nividiaAgACI[[2]])
errSeqsFannie <- cbind(fannieStableGridGammas[[6]],fannieStableGridGammas[[3]],fannieMVP[[2]],fannieAgACI[[2]])

amdPlot <- makeCoveragePlot(errSeqsAMD,allDates[[2]],alpha,cols=c("DtACI","Constant Baseline","MVP","AgACI"),colors=c(4,7,3,2),timeCutoff=500)
amdPlot <- amdPlot + scale_colour_manual(name="Method",values = c("DtACI"=4,"AgACI"=2,"MVP"=3,"Constant Baseline"=7),
                                                             labels=c("DtACI","AgACI","MVP","Constant Baseline"),
                                                             aesthetics = c("color")) + ylim(0.8,0.965) +
                    theme(axis.text.x = element_blank(),axis.ticks.x = element_blank(),axis.title.x = element_blank(),
                         axis.text.y = element_blank(),axis.ticks.y = element_blank(),axis.title.y = element_blank()) +
                    theme(legend.key.width = unit(1,"cm"))
bbPlot <- makeCoveragePlot(errSeqsBB,allDates[[3]],alpha,cols=c("DtACI","Constant Baseline","MVP","AgACI"),colors=c(4,7,3,2),timeCutoff=500)
bbPlot <- bbPlot + scale_colour_manual(name="Method",values = c("DtACI"=4,"AgACI"=2,"MVP"=3,"Constant Baseline"=7),
                                                             labels=c("DtACI","AgACI","MVP","Constant Baseline"),
                                                             aesthetics = c("color"))  + ylim(0.8,0.965)
nvidiaPlot <- makeCoveragePlot(errSeqsNvidia,allDates[[1]],alpha,cols=c("DtACI","Constant Baseline","MVP","AgACI"),colors=c(4,7,3,2),timeCutoff=500)
nvidiaPlot <- nvidiaPlot + scale_colour_manual(name="Method",values = c("DtACI"=4,"AgACI"=2,"MVP"=3,"Constant Baseline"=7),
                                                             labels=c("DtACI","AgACI","MVP","Split"),
                                                             aesthetics = c("color")) + ylim(0.8,0.965) +
                            theme(axis.text.x = element_blank(),axis.ticks.x = element_blank(),axis.title.x = element_blank())
fanniePlot <- makeCoveragePlot(errSeqsFannie,allDates[[4]],alpha,cols=c("DtACI","Constant Baseline","MVP","AgACI"),colors=c(4,7,3,2),timeCutoff=500)
fanniePlot <- fanniePlot + scale_colour_manual(name="Method",values = c("DtACI"=4,"AgACI"=2,"MVP"=3,"Constant Baseline"=7),
                                                             labels=c("DtACI","AgACI","MVP","Constant Baseline"),
                                                             aesthetics = c("color")) + ylim(0.8,0.965)  + 
                            theme(axis.text.y = element_blank(),axis.ticks.y = element_blank(),axis.title.y = element_blank())

legend <- get_legend(amdPlot+theme(legend.position = "top",legend.text=element_text(size=60),legend.title=element_blank(),
                                   legend.key.width = unit(3,"line")))
blankplot <- ggplot()+geom_blank(aes(1,1)) + cowplot::theme_nothing()

nvidiaPlot <- nvidiaPlot +  annotate('text',x=as.Date("2005-06-01"), y=0.958, label="Nvidia",size=18,fontface="bold") + theme(legend.position = "none")
amdPlot <- amdPlot  +  annotate('text',x=as.Date("1997-09-20"), y=0.958, label="AMD",size=18,fontface="bold") + theme(legend.position = "none")
bbPlot <- bbPlot +  annotate('text',x=as.Date("2006-09-01"), y=0.958, label="BlackBerry",size=18,fontface="bold") + theme(legend.position = "none")
fanniePlot <- fanniePlot +  annotate('text',x=as.Date("2000-02-20"), y=0.958, label="Fannie Mae",size=18,fontface="bold") + theme(legend.position = "none")

pdf("Figures/VolatilityCoverage.pdf", width=30, height = 20)
grid.arrange(legend,blankplot,nvidiaPlot,amdPlot,bbPlot,fanniePlot,ncol=2,widths=c(3.5,3),heights = c(0.4,2,2.5))
dev.off()

Warning message:
“Removed 244 rows containing missing values or values outside the scale range
(`geom_line()`).”
Warning message in get_plot_component(plot, "guide-box"):
“Multiple components found; returning the first one. To return all, use `return_all = TRUE`.”
Warning message:
“Removed 239 rows containing missing values or values outside the scale range
(`geom_line()`).”
Warning message:
“Removed 244 rows containing missing values or values outside the scale range
(`geom_line()`).”
Warning message:
“Removed 268 rows containing missing values or values outside the scale range
(`geom_line()`).”
Warning message:
“Removed 244 rows containing missing values or values outside the scale range
(`geom_line()`).”


pdf 
  2

In [21]:
### Make qq plots

gg_qq_empirical <- function(a, b, quantiles=seq(0.05,0.95,0.1)) 
{
  ggplot(mapping = aes(x = quantile(a, quantiles), 
                       y = quantile(b, quantiles))) + 
    geom_point(size = 6) +
    geom_abline(aes(slope = 1, intercept = 0), linetype = 2, size = 2) +
    labs(x = paste(deparse(substitute(a)), "quantiles"), 
         y = paste(deparse(substitute(b)), "quantiles"))
}

plotBerQQ <- function(errs,alpha=0.1,smoothingWidth=500, nber = 100000){
  
    T <- length(errs)
    
    if(smoothingWidth>0){
        errs <- abs(sapply(1:T,function(x){localMean(errs,x,bandwidth=1,bottom=smoothingWidth)}) - alpha)[-(1:smoothingWidth)]
      }
    print(max(errs))
    
    bernSeq <- rbinom(nber,1,alpha)
    smoothedBer <- abs(sapply(1:nber,function(x){localMean(bernSeq,x,bandwidth=1,bottom=smoothingWidth)}) - alpha)[-(1:smoothingWidth)]
    
    p <- gg_qq_empirical(errs, smoothedBer) + xlab("Quantiles of DtACI Error Gaps") + ylab("Quantiles of Bern. Error Gaps")
    p <- p + ylim(0,0.03) + xlim(0,0.03) + theme_bw() + theme(axis.text=element_text(size=40),axis.title=element_text(size=40,face="plain")) 
    p <- p 

    return(p)
}

amdQQ <- plotBerQQ(amdStableGridGammas[[6]]) +  annotate('text',x=0.001, y=0.029, label="AMD",size=18,fontface="bold") 
nividiaQQ <- plotBerQQ(nividiaStableGridGammas[[6]]) +  annotate('text',0.0015, y=0.029, label="Nvidia",size=18,fontface="bold") 
blackberryQQ <- plotBerQQ(blackberryStableGridGammas[[6]]) +  annotate('text',0.0036, y=0.029, label="BlackBerry",size=18,fontface="bold")
fannieQQ <- plotBerQQ(fannieStableGridGammas[[6]]) +  annotate('text',x=0.0037, y=0.029, label="Fannie Mae",size=18,fontface="bold")

amdQQ <- amdQQ + theme(axis.text.x = element_blank(),axis.ticks.x = element_blank(),axis.title.x = element_blank(),
                         axis.text.y = element_blank(),axis.ticks.y = element_blank(),axis.title.y = element_blank())
nividiaQQ <- nividiaQQ + theme(axis.text.x = element_blank(),axis.ticks.x = element_blank(),axis.title.x = element_blank())
fannieQQ <- fannieQQ + theme(axis.text.y = element_blank(),axis.ticks.y = element_blank(),axis.title.y = element_blank())

pdf("Figures/BernComparison.pdf", width=30, height = 20)
options(repr.plot.width=30, repr.plot.height=20)

grid.arrange(nividiaQQ,amdQQ,blackberryQQ,fannieQQ,ncol=2,widths=c(3.5,3))
dev.off()

[1] 0.03
[1] 0.028
[1] 0.028
[1] 0.034


pdf 
  2

In [22]:

amdQQ <- plotBerQQ(amdStableGridGammasBad[[6]]) +  annotate('text',x=0.001, y=0.029, label="AMD",size=18,fontface="bold") 
nividiaQQ <- plotBerQQ(nividiaStableGridGammasBad[[6]]) +  annotate('text',0.0015, y=0.029, label="Nvidia",size=18,fontface="bold") 
blackberryQQ <- plotBerQQ(blackberryStableGridGammasBad[[6]]) +  annotate('text',0.0036, y=0.029, label="BlackBerry",size=18,fontface="bold")
fannieQQ <- plotBerQQ(fannieStableGridGammasBad[[6]]) +  annotate('text',x=0.0037, y=0.029, label="Fannie Mae",size=18,fontface="bold") 

amdQQ <- amdQQ + theme(axis.text.x = element_blank(),axis.ticks.x = element_blank(),axis.title.x = element_blank(),
                         axis.text.y = element_blank(),axis.ticks.y = element_blank(),axis.title.y = element_blank())
nividiaQQ <- nividiaQQ + theme(axis.text.x = element_blank(),axis.ticks.x = element_blank(),axis.title.x = element_blank())
fannieQQ <- fannieQQ + theme(axis.text.y = element_blank(),axis.ticks.y = element_blank(),axis.title.y = element_blank())

pdf("Figures/BernComparisonBad.pdf", width=30, height = 20)
#options(repr.plot.width=30, repr.plot.height=20)

grid.arrange(nividiaQQ,amdQQ,blackberryQQ,fannieQQ,ncol=2,widths=c(3.5,3))
dev.off()

[1] 0.036
[1] 0.028
[1] 0.03
[1] 0.034


pdf 
  2

### Fixed versus dynamic eta

In [24]:
### Compute results for a dynamic eta

amdStableGridGammasDEta <- conformalAdaptStable(amdBetas,alpha,gammaGrid,sigma=1/1000,eta=2.72,etaAdapt=TRUE)
blackberryStableGridGammasDEta <- conformalAdaptStable(blackberryBetas,alpha,gammaGrid,sigma=1/1000,eta=2.72,etaAdapt=TRUE)
nividiaStableGridGammasDEta <- conformalAdaptStable(nividiaBetas,alpha,gammaGrid,sigma=1/1000,eta=2.72,etaAdapt=TRUE)
fannieStableGridGammasDEta <- conformalAdaptStable(fannieBetas,alpha,gammaGrid,sigma=1/1000,eta=2.72,etaAdapt=TRUE)

amdStableGridGammasBadDEta <- conformalAdaptStable(amdBadBetas,alpha,gammaGrid,sigma=1/1000,eta=2.72,etaAdapt=TRUE)
blackberryStableGridGammasBadDEta <- conformalAdaptStable(blackberryBadBetas,alpha,gammaGrid,sigma=1/1000,eta=2.72,etaAdapt=TRUE)
nividiaStableGridGammasBadDEta <- conformalAdaptStable(nividiaBadBetas,alpha,gammaGrid,sigma=1/1000,eta=2.72,etaAdapt=TRUE)
fannieStableGridGammasBadDEta <- conformalAdaptStable(fannieBadBetas,alpha,gammaGrid,sigma=1/1000,eta=2.72,etaAdapt=TRUE)


In [25]:
library(latex2exp)

allDates <- list(nividia$Date[1:length(nividiaStableGridGammas[[1]])],
                 amd$Date[1:length(amdStableGridGammas[[1]])],
                 blackberry$Date[1:length(blackberryStableGridGammas[[1]])],
                 fannie$Date[1:length(fannieStableGridGammas[[1]])])
for(d in 1:length(allDates)){
  allDates[[d]] <- rev(allDates[[d]])
  allDates[[d]] <- as.Date(allDates[[d]], '%m/%d/%y')
}

errSeqsAMD <- cbind(amdStableGridGammas[[6]],amdStableGridGammasDEta[[6]])
errSeqsBB <- cbind(blackberryStableGridGammas[[6]],blackberryStableGridGammasDEta[[6]])
errSeqsNvidia <- cbind(nividiaStableGridGammas[[6]],nividiaStableGridGammasDEta[[6]])
errSeqsFannie <- cbind(fannieStableGridGammas[[6]],fannieStableGridGammasDEta[[6]])

amdPlot <- makeCoveragePlot(errSeqsAMD,allDates[[2]],alpha,cols=c("Fixed","Dynamic"),colors=c(4,6),timeCutoff=500,lineTyping=TRUE,alphaC=1)
amdPlot <- amdPlot + scale_colour_manual(name="Method",values = c("Fixed"=4,"Dynamic"=6),
                                                             labels=c(TeX("DtACI Fixed $\\eta$"),TeX("DtACI Variable $\\eta$")),
                                                             aesthetics = c("color")) +
                    scale_linetype_manual(name="Method",values = c("Fixed" = "dashed", "Dynamic" = "solid"),
                                                             labels=c(TeX("DtACI Fixed $\\eta$"),TeX("DtACI Variable $\\eta$"))) + ylim(0.8,0.965) +
                    theme(axis.text.x = element_blank(),axis.ticks.x = element_blank(),axis.title.x = element_blank(),
                         axis.text.y = element_blank(),axis.ticks.y = element_blank(),axis.title.y = element_blank()) +
                    theme(legend.key.width = unit(1,"cm"))
bbPlot <- makeCoveragePlot(errSeqsBB,allDates[[3]],alpha,cols=c("Fixed","Dynamic"),colors=c(4,6),timeCutoff=500,lineTyping=TRUE,alphaC=1)
bbPlot <- bbPlot + scale_colour_manual(name="Method",values = c("Fixed"=4,"Dynamic"=6),
                                                             labels=c(TeX("DtACI Fixed $\\eta$"),TeX("DtACI Dynamic $\\eta$")),
                                                             aesthetics = c("color")) +
                    scale_linetype_manual(name="Method",values = c("Fixed" = "dashed", "Dynamic" = "solid"),
                                                             labels=c(TeX("DtACI Fixed $\\eta$"),TeX("DtACI Dynamic $\\eta$"))) + ylim(0.8,0.965) 
nvidiaPlot <- makeCoveragePlot(errSeqsNvidia,allDates[[1]],alpha,cols=c("Fixed","Dynamic"),colors=c(4,6),timeCutoff=500,lineTyping=TRUE,alphaC=1)
nvidiaPlot <- nvidiaPlot + scale_colour_manual(name="Method",values = c("Fixed"=4,"Dynamic"=6),
                                                             labels=c(TeX("DtACI Fixed $\\eta$"),TeX("DtACI Dynamic $\\eta$")),
                                                             aesthetics = c("color")) + 
                            scale_linetype_manual(name="Method",values = c("Fixed" = "dashed", "Dynamic" = "solid"),
                                                             labels=c(TeX("DtACI Fixed $\\eta$"),TeX("DtACI Dynamic $\\eta$"))) + ylim(0.8,0.965) +
                            theme(axis.text.x = element_blank(),axis.ticks.x = element_blank(),axis.title.x = element_blank())
fanniePlot <- makeCoveragePlot(errSeqsFannie,allDates[[4]],alpha,cols=c("Fixed","Dynamic"),colors=c(4,6),timeCutoff=500,lineTyping=TRUE,alphaC=1)
fanniePlot <- fanniePlot + scale_colour_manual(name="Method",values = c("Fixed"=4,"Dynamic"=6),
                                                             labels=c(TeX("DtACI Fixed $\\eta$"),TeX("DtACI Dynamic $\\eta$")),
                                                             aesthetics = c("color")) + 
                            scale_linetype_manual(name="Method",values = c("Fixed" = "dashed", "Dynamic" = "solid"),
                                                             labels=c(TeX("DtACI Fixed $\\eta$"),TeX("DtACI Dynamic $\\eta$"))) + ylim(0.8,0.965) +
                            theme(axis.text.y = element_blank(),axis.ticks.y = element_blank(),axis.title.y = element_blank())

legend <- get_legend(amdPlot+theme(legend.position = "top",legend.text=element_text(size=60),legend.title=element_blank(),
                                   legend.key.width = unit(3,"line")))
blankplot <- ggplot()+geom_blank(aes(1,1)) + cowplot::theme_nothing()

nvidiaPlot <- nvidiaPlot +  annotate('text',x=as.Date("2005-09-01"), y=0.958, label="Nvidia",size=18,fontface="bold") + theme(legend.position = "none")
amdPlot <- amdPlot  +  annotate('text',x=as.Date("1998-01-20"), y=0.958, label="AMD",size=18,fontface="bold") + theme(legend.position = "none")
bbPlot <- bbPlot +  annotate('text',x=as.Date("2006-10-01"), y=0.958, label="BlackBerry",size=18,fontface="bold") + theme(legend.position = "none")
fanniePlot <- fanniePlot +  annotate('text',x=as.Date("2000-04-20"), y=0.958, label="Fannie Mae",size=18,fontface="bold") + theme(legend.position = "none")

pdf("Figures/VolatilityCoverageVariableEta.pdf", width=30, height = 20)
grid.arrange(legend,blankplot,nvidiaPlot,amdPlot,bbPlot,fanniePlot,ncol=2,widths=c(3.5,3),heights = c(0.4,2,2.5))
dev.off()

Warning message in get_plot_component(plot, "guide-box"):
“Multiple components found; returning the first one. To return all, use `return_all = TRUE`.”


pdf 
  2

In [26]:
errSeqsAMD <- cbind(amdStableGridGammasBad[[6]],amdStableGridGammasBadDEta[[6]])
errSeqsBB <- cbind(blackberryStableGridGammasBad[[6]],blackberryStableGridGammasBadDEta[[6]])
errSeqsNvidia <- cbind(nividiaStableGridGammasBad[[6]],nividiaStableGridGammasBadDEta[[6]])
errSeqsFannie <- cbind(fannieStableGridGammasBad[[6]],fannieStableGridGammasBadDEta[[6]])

amdPlotBad <- makeCoveragePlot(errSeqsAMD,allDates[[2]],alpha,cols=c("Fixed","Dynamic"),colors=c(4,6),timeCutoff=500,lineTyping=TRUE,alphaC=1)
amdPlotBad <- amdPlotBad + scale_colour_manual(name="Method",values = c("Fixed"=4,"Dynamic"=6),
                                                             labels=c(TeX("DtACI Fixed $\\eta$"),TeX("DtACI Variable $\\eta$")),
                                                             aesthetics = c("color")) +
                    scale_linetype_manual(name="Method",values = c("Fixed" = "dashed", "Dynamic" = "solid"),
                                                             labels=c(TeX("DtACI Fixed $\\eta$"),TeX("DtACI Variable $\\eta$"))) + ylim(0.8,0.965) +
                    theme(axis.text.x = element_blank(),axis.ticks.x = element_blank(),axis.title.x = element_blank(),
                         axis.text.y = element_blank(),axis.ticks.y = element_blank(),axis.title.y = element_blank()) +
                    theme(legend.key.width = unit(1,"cm"))
bbPlotBad <- makeCoveragePlot(errSeqsBB,allDates[[3]],alpha,cols=c("Fixed","Dynamic"),colors=c(4,6),timeCutoff=500,lineTyping=TRUE,alphaC=1)
bbPlotBad <- bbPlotBad + scale_colour_manual(name="Method",values = c("Fixed"=4,"Dynamic"=6),
                                                             labels=c(TeX("DtACI Fixed $\\eta$"),TeX("DtACI Dynamic $\\eta$")),
                                                             aesthetics = c("color")) +
                    scale_linetype_manual(name="Method",values = c("Fixed" = "dashed", "Dynamic" = "solid"),
                                                             labels=c(TeX("DtACI Fixed $\\eta$"),TeX("DtACI Dynamic $\\eta$"))) +  ylim(0.8,0.965)
nvidiaPlotBad <- makeCoveragePlot(errSeqsNvidia,allDates[[1]],alpha,cols=c("Fixed","Dynamic"),colors=c(4,6),timeCutoff=500,lineTyping=TRUE,alphaC=1)
nvidiaPlotBad <- nvidiaPlotBad + scale_colour_manual(name="Method",values = c("Fixed"=4,"Dynamic"=6),
                                                             labels=c(TeX("DtACI Fixed $\\eta$"),TeX("DtACI Dynamic $\\eta$")),
                                                             aesthetics = c("color")) + 
                            scale_linetype_manual(name="Method",values = c("Fixed" = "dashed", "Dynamic" = "solid"),
                                                             labels=c(TeX("DtACI Fixed $\\eta$"),TeX("DtACI Dynamic $\\eta$"))) + ylim(0.8,0.965) +
                            theme(axis.text.x = element_blank(),axis.ticks.x = element_blank(),axis.title.x = element_blank())
fanniePlotBad <- makeCoveragePlot(errSeqsFannie,allDates[[4]],alpha,cols=c("Fixed","Dynamic"),colors=c(4,6),timeCutoff=500,lineTyping=TRUE,alphaC=1)
fanniePlotBad <- fanniePlotBad + scale_colour_manual(name="Method",values = c("Fixed"=4,"Dynamic"=6),
                                                             labels=c(TeX("DtACI Fixed $\\eta$"),TeX("DtACI Dynamic $\\eta$")),
                                                             aesthetics = c("color")) + 
                            scale_linetype_manual(name="Method",values = c("Fixed" = "dashed", "Dynamic" = "solid"),
                                                             labels=c(TeX("DtACI Fixed $\\eta$"),TeX("DtACI Dynamic $\\eta$"))) +  ylim(0.8,0.965) +
                            theme(axis.text.y = element_blank(),axis.ticks.y = element_blank(),axis.title.y = element_blank())

legend <- get_legend(amdPlot+theme(legend.position = "top",legend.text=element_text(size=60),legend.title=element_blank(),
                                   legend.key.width = unit(3,"line")))
blankplot <- ggplot()+geom_blank(aes(1,1)) + cowplot::theme_nothing()

nvidiaPlotBad <- nvidiaPlotBad +  annotate('text',x=as.Date("2005-09-01"), y=0.958, label="Nvidia",size=18,fontface="bold") + theme(legend.position = "none")
amdPlotBad <- amdPlotBad  +  annotate('text',x=as.Date("1998-01-20"), y=0.958, label="AMD",size=18,fontface="bold") + theme(legend.position = "none")
bbPlotBad <- bbPlotBad +  annotate('text',x=as.Date("2006-10-01"), y=0.958, label="BlackBerry",size=18,fontface="bold") + theme(legend.position = "none")
fanniePlotBad <- fanniePlotBad +  annotate('text',x=as.Date("2000-04-20"), y=0.958, label="Fannie Mae",size=18,fontface="bold") + theme(legend.position = "none")

pdf("Figures/VolatilityCoverageVariableEtaBad.pdf", width=30, height = 20)
grid.arrange(legend,blankplot,nvidiaPlotBad,amdPlotBad,bbPlotBad,fanniePlotBad,ncol=2,widths=c(3.5,3),heights = c(0.4,2,2.5))
dev.off()

Warning message in get_plot_component(plot, "guide-box"):
“Multiple components found; returning the first one. To return all, use `return_all = TRUE`.”


pdf 
  2

### Plot prices over time

In [27]:
pricesAMD <- rev(amd$Open)
returnsAMD <- sapply(2:length(pricesAMD),function(x){(pricesAMD[x] - pricesAMD[x-1])/pricesAMD[x-1]})
returnsAMD <- returnsAMD[(length(returnsAMD)-7500):length(returnsAMD)]
pricesBlackberry<- rev(blackberry$Open)
returnsBlackberry <- sapply(2:length(pricesBlackberry),function(x){(pricesBlackberry[x] - pricesBlackberry[x-1])/pricesBlackberry[x-1]})
pricesNividia <- rev(nividia$Open)
returnsNividia <- sapply(2:length(pricesNividia),function(x){(pricesNividia[x] - pricesNividia[x-1])/pricesNividia[x-1]})
pricesFannie <- rev(fannie$Open)
returnsFannie <- sapply(2:length(pricesFannie),function(x){(pricesFannie[x] - pricesFannie[x-1])/pricesFannie[x-1]})
returnsFannie <- returnsFannie[(length(returnsFannie)-7500):length(returnsFannie)]


allThingsToPlot <- list(pricesNividia[(length(pricesNividia)-length(nividiaStableGridGammas[[1]])-1249):length(pricesNividia)],
                        pricesAMD[(length(pricesAMD)-length(amdStableGridGammas[[1]])-1249):length(pricesAMD)],
                        pricesBlackberry[(length(pricesBlackberry)-length(blackberryStableGridGammas[[1]])-1249):length(pricesBlackberry)],
                        pricesFannie[(length(pricesFannie)-length(fannieStableGridGammas[[1]])-1249):length(pricesFannie)])


allDates <- list(nividia$Date[1:(length(nividiaStableGridGammas[[1]])+1250)],
                 amd$Date[1:(length(amdStableGridGammas[[1]])+1250)],
                 blackberry$Date[1:(length(blackberryStableGridGammas[[1]])+1250)],
                 fannie$Date[1:(length(fannieStableGridGammas[[1]])+1250)])
for(d in 1:length(allDates)){
  allDates[[d]] <- rev(allDates[[d]])
  allDates[[d]] <- as.Date(allDates[[d]], '%m/%d/%y')
}

pricePlots <- list()
for(i in 1:4){
  temp <- data.frame(prices = allThingsToPlot[[i]], dates = as.Date(allDates[[i]]))
  pricePlots[[i]] <- ggplot(temp,aes(dates)) + geom_line(aes(y=prices, colour="black"),size=4) + 
    scale_colour_manual(name = "Method", values =c("black"="black"), labels = c(""))  + theme_bw() +
    theme(legend.position = "none") + xlab("Time") + ylab("Open Price") +
    theme(axis.text=element_text(size=40),axis.title=element_text(size=50,face="plain"))
}

pricePlots[[1]] <- pricePlots[[1]] + theme(axis.text.x = element_blank(),axis.ticks.x = element_blank(),axis.title.x = element_blank())
pricePlots[[2]] <- pricePlots[[2]] + theme(axis.text.x = element_blank(),axis.ticks.x = element_blank(),axis.title.x = element_blank())
pricePlots[[4]] <- pricePlots[[4]] + ylim(0,100)

pricePlots[[1]] <- pricePlots[[1]] +  annotate('text',x=as.Date("2000-03-01"), y=640, label="Nvidia",size=18,fontface="bold")
pricePlots[[2]] <- pricePlots[[2]]  +  annotate('text',x=as.Date("1992-06-01"), y=140, label="AMD",size=18,fontface="bold")
pricePlots[[3]] <- pricePlots[[3]] +  annotate('text',x=as.Date("2001-08-01"), y=147, label="BlackBerry",size=18,fontface="bold")
pricePlots[[4]] <- pricePlots[[4]] +  annotate('text',x=as.Date("1994-12-20"), y=100, label="Fannie Mae",size=18,fontface="bold")


pdf("Figures/stockPrices.pdf", width=30, height = 20)
grid.arrange(pricePlots[[1]],pricePlots[[2]],pricePlots[[3]],pricePlots[[4]],ncol=2,widths=c(3,3),heights=c(2,2.5))
dev.off()

pdf 
  2